In [ ]:
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from collections import Counter

In [ ]:
import math
import scipy
import shapely
import datetime
import scipy
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_pickle(f"/content/drive/MyDrive/training_data_grids_final.pkl")
data_lags_nn_queen_less_rook = pd.read_pickle(f"/content/drive/MyDrive/training_data_grids_nn_queen_less_rook.pkl")
data_lags_nn_rook = pd.read_pickle(f"/content/drive/MyDrive/training_data_grids_nn_rook.pkl")

data = data.drop([x for x in data.columns if 'neighb' in x or 'edge' in x or 'geom' in x], axis=1)
data = pd.concat([data[[c for c in data.columns if "nn" not in c]], data_lags_nn_queen_less_rook, data_lags_nn_rook], axis=1)

In [ ]:
data.shape

(8760, 318)

In [ ]:
data['day'] = data['time'].apply(lambda x: x.day)
data['day_of_year'] = data['time'].apply(lambda x: x.timetuple().tm_yday)
data['month'] = data['time'].apply(lambda x: x.month)
data['week'] = data['time'].apply(lambda x: x.isocalendar()[1])
data['weekday'] = data['time'].apply(lambda x: x.weekday())

In [ ]:
##  monitoring stations
grid_stations = gpd.read_file(f"/content/drive/MyDrive/training_data_grid_stations.shp")
##  outcome of SO2 data
out_so2 = pd.read_pickle(f"/content/drive/MyDrive/training_data_out_so2.pkl")

In [ ]:
def dist(x, y):
    lat1 = math.radians(x[0])
    lon1 = math.radians(x[1])
    lat2 = math.radians(y[0])
    lon2 = math.radians(y[1])
    R = 6373.0
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return round(distance, 4)

In [ ]:
##  Create Lagged Features of Surface Monitored SO2  ##

weighted_so2 = []
var_list = ['min_so2','mean_so2','max_so2',
            'min_so2_4hr','mean_so2_4hr','max_so2_4hr',
            'min_so2_8hr','mean_so2_8hr','max_so2_8hr',
            'min_so2_12hr','mean_so2_12hr','max_so2_12hr',
            'min_so2_24hr','mean_so2_24hr','max_so2_24hr',
            'min_so2_48hr','mean_so2_48hr','max_so2_48hr']

decay_factor = 0.1      #  multiplier to control the rate of exponential decay by distance

for n in range(data[['id','lon','lat']].drop_duplicates().shape[0]):

    #  generate distance matrix between stations

    tem = data[['id','lon','lat']].drop_duplicates().iloc[n]
    tem_coords_ref = tem[['lat','lon']].values.tolist()
    distances = [dist(tem_coords_ref, z) for z in grid_stations[['latitude','longitude']].values.tolist()]
    tem_dist = pd.DataFrame({"id": grid_stations["id_right"].values.tolist(), "station_name": grid_stations["station_name"].values.tolist(), "distances": distances})

    #  include influence from nearby stations within a 10 km buffer,
    #  exclude the in-situ / local-grid station to prevent data leakage in modelling

    tem_dist = tem_dist[tem_dist['distances'] <= 10]
    tem_leak = tem_dist[tem_dist['distances'] <= np.sqrt(0.275**2 + 0.275**2)]

    if tem_dist.shape[0] > 0:

        #  compute inverse-distance weighting for the influence (SO2 monitored observations) from nearby stations (spatial lags)

        tem_dist['weighting'] = 1 / (tem_dist['distances'] / tem_dist['distances'].sum())
        tem_dist['weighting'] = tem_dist['weighting'] / tem_dist['weighting'].sum()
        tem_out_so2 = out_so2.merge(tem_dist, left_on = ['station_name','id_right'], right_on = ['station_name','id'], how="inner")

        if tem_leak.shape[0] > 0:
            tem_out_so2['min_so2'] = tem_out_so2.apply(lambda x: x['min_so2'] if x['station_name'] != tem_leak['station_name'].values[0] else np.nan, axis=1)
            tem_out_so2['mean_so2'] = tem_out_so2.apply(lambda x: x['mean_so2'] if x['station_name'] != tem_leak['station_name'].values[0] else np.nan, axis=1)
            tem_out_so2['max_so2'] = tem_out_so2.apply(lambda x: x['max_so2'] if x['station_name'] != tem_leak['station_name'].values[0] else np.nan, axis=1)

        for var in var_list:
            tem_out_so2['weighted_' + var] = tem_out_so2[var] * tem_out_so2['weighting']

        tem_out_so2_agg = tem_out_so2.groupby(["day_time"]).agg({k: [np.nansum] for k in ['weighted_' + x for x in var_list]}).reset_index()
        tem_out_so2_agg.columns = ["time"] + ['weighted_' + x for x in var_list]

        #  apply log-transformation to the SO2 data
        #  apply exponential decay function by distance to the SO2 data, hence influence from stations farther away will diminish

        for var in var_list:
            tem_out_so2_agg['weighted_' + var] = np.log(tem_out_so2_agg['weighted_' + var] + 0.001)
            tem_out_so2_agg['weighted_' + var] = tem_out_so2_agg['weighted_' + var] * np.exp(-1 * decay_factor * tem_dist['distances'].min())

        tem_out_so2_agg['id'] = data['id'].drop_duplicates().iloc[n]

        #  create temporal lags for monitored SO2 at local-grid station up to past 6 days

        for lags in range(6):
            tem_out_so2_agg['weighted_min_so2_lag_' + str(lags+1)] = tem_out_so2_agg['weighted_min_so2'].shift(lags+1)
            tem_out_so2_agg['weighted_mean_so2_lag_' + str(lags+1)] = tem_out_so2_agg['weighted_mean_so2'].shift(lags+1)
            tem_out_so2_agg['weighted_max_so2_lag_' + str(lags+1)] = tem_out_so2_agg['weighted_max_so2'].shift(lags+1)

        tem_out_so2_agg = tem_out_so2_agg[tem_out_so2_agg['time'] >= datetime.datetime.date(datetime.datetime.strptime('2022-01-01', "%Y-%m-%d"))]

        weighted_so2.append(tem_out_so2_agg)

weighted_so2 = pd.concat(weighted_so2)

data = data.merge(weighted_so2, on=['id','time'], how='left')

In [ ]:
##  binary outcome variable for SO2-clean vs SO2-polluted days
data['SO2_above_0'] = data['mean_so2'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
data.shape

(8760, 360)

In [ ]:
def cross_validate_fit_binary_classification(model, X_train, y_train):

    accuracy = []
    precision = []
    recall = []
    f1score = []

    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        X_train_cv, y_train_cv = X_train[train_index], y_train.values[train_index]
        X_test_cv, y_test_cv = X_train[test_index], y_train.values[test_index]

        model.fit(X_train, y_train)

        binary_preds_train = model.predict(X_train_cv)
        binary_preds_test = model.predict(X_test_cv)

        m1 = (accuracy_score(y_test_cv, binary_preds_test), accuracy_score(y_train_cv, binary_preds_train))
        m2 = (precision_score(y_test_cv, binary_preds_test, average="weighted"), precision_score(y_train_cv, binary_preds_train, average="weighted"))
        m3 = (recall_score(y_test_cv, binary_preds_test, average="weighted"), recall_score(y_train_cv, binary_preds_train, average="weighted"))
        m4 = (f1_score(y_test_cv, binary_preds_test, average="weighted"), f1_score(y_train_cv, binary_preds_train, average="weighted"))

        accuracy.append(m1)
        precision.append(m2)
        recall.append(m3)
        f1score.append(m4)

    return accuracy, precision, recall, f1score

In [ ]:
def cross_validate_fit_regression(model, X_train, y_train):

    m1a, m2a, m3a, m4a, m5a, m6a, m7a = [], [], [], [], [], [], []
    m1b, m2b, m3b, m4b, m5b, m6b, m7b = [], [], [], [], [], [], []

    kf = KFold(n_splits=10, random_state=42, shuffle=True)
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        X_train_cv, y_train_cv = X_train[train_index], y_train.values[train_index]
        X_test_cv, y_test_cv = X_train[test_index], y_train.values[test_index]

        model.fit(X_train, y_train)

        gbmlogmean_preds_train = model.predict(X_train_cv)
        gbmlogmean_preds_test = model.predict(X_test_cv)

        print("Train R2: " + str(r2_score(y_train_cv, gbmlogmean_preds_train)))
        print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train_cv).flatten(), gbmlogmean_preds_train)[0]))
        print("Train MAE: " + str(mean_absolute_error(y_train_cv, gbmlogmean_preds_train)))
        print("Train MedAE: " + str(median_absolute_error(y_train_cv, gbmlogmean_preds_train)))
        print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train_cv) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001)))
        print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train_cv) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001)))
        print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train_cv) - np.exp(gbmlogmean_preds_train)), 0.75)))
        print("\n")
        print("Test R2: " + str(r2_score(y_test_cv, gbmlogmean_preds_test)))
        print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test_cv).flatten(), gbmlogmean_preds_test)[0]))
        print("Test MAE: " + str(mean_absolute_error(y_test_cv, gbmlogmean_preds_test)))
        print("Test MedAE: " + str(median_absolute_error(y_test_cv, gbmlogmean_preds_test)))
        print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test_cv) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001)))
        print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test_cv) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001)))
        print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test_cv) - np.exp(gbmlogmean_preds_test)), 0.75)))
        print("\n")

        m1a.append(r2_score(y_test_cv, gbmlogmean_preds_test))
        m2a.append(scipy.stats.pearsonr(np.array(y_test_cv).flatten(), gbmlogmean_preds_test)[0])
        m3a.append(mean_absolute_error(y_test_cv, gbmlogmean_preds_test))
        m4a.append(median_absolute_error(y_test_cv, gbmlogmean_preds_test))
        m5a.append(mean_absolute_error(np.exp(y_test_cv) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001))
        m6a.append(median_absolute_error(np.exp(y_test_cv) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001))
        m7a.append(np.quantile(abs(np.exp(y_test_cv) - np.exp(gbmlogmean_preds_test)), 0.75))

        m1b.append(r2_score(y_train_cv, gbmlogmean_preds_train))
        m2b.append(scipy.stats.pearsonr(np.array(y_train_cv).flatten(), gbmlogmean_preds_train)[0])
        m3b.append(mean_absolute_error(y_train_cv, gbmlogmean_preds_train))
        m4b.append(median_absolute_error(y_train_cv, gbmlogmean_preds_train))
        m5b.append(mean_absolute_error(np.exp(y_train_cv) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001))
        m6b.append(median_absolute_error(np.exp(y_train_cv) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001))
        m7b.append(np.quantile(abs(np.exp(y_train_cv) - np.exp(gbmlogmean_preds_train)), 0.75))

    return m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
            m1b, m2b, m3b, m4b, m5b, m6b, m7b

####  **CONTROL 1) Lasso Regression： only Meteorological and Land-Use Land-Cover features**

#####  **Binary Classification (Cross-Validation):**

In [ ]:
y = data['SO2_above_0']
X = data.iloc[:,6:323]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
lasso_logistic = Pipeline([("imputer", KNNImputer(n_neighbors=4, weights='uniform')),
                           ("regressor", LogisticRegression(penalty='l1', solver='liblinear'))])

accuracy, precision, recall, f1score = cross_validate_fit_binary_classification(lasso_logistic , X_train, y_train)

In [ ]:
print(np.mean([accuracy[n][0] for n in range(10)]))
print(np.mean([precision[n][0] for n in range(10)]))
print(np.mean([recall[n][0] for n in range(10)]))
print(np.mean([f1score[n][0] for n in range(10)]))

0.8296254330548198
0.7984864869667627
0.8296254330548198
0.7980414422186178


#####  **Binary Classification (Training & Testing):**

In [ ]:
lasso_logistic = Pipeline([("imputer", KNNImputer(n_neighbors=4, weights='uniform')),
                           ("regressor", LogisticRegression(penalty='l1', solver='liblinear'))])
lasso_logistic.fit(X_train, y_train)

Pipeline(steps=[('imputer', KNNImputer(n_neighbors=4)),
                ('regressor',
                 LogisticRegression(penalty='l1', solver='liblinear'))])

In [ ]:
lasso_logistic_preds_train = lasso_logistic.predict(X_train)
lasso_logistic_preds_test = lasso_logistic.predict(X_test)

In [ ]:
print(classification_report(y_train, lasso_logistic_preds_train))

              precision    recall  f1-score   support

           0       0.56      0.22      0.32      1252
           1       0.85      0.96      0.90      5756

    accuracy                           0.83      7008
   macro avg       0.71      0.59      0.61      7008
weighted avg       0.80      0.83      0.80      7008



In [ ]:
confusion_matrix(y_train, lasso_logistic_preds_train)

array([[ 277,  975],
       [ 215, 5541]])

In [ ]:
print(classification_report(y_test, lasso_logistic_preds_test))

              precision    recall  f1-score   support

           0       0.51      0.19      0.28       313
           1       0.84      0.96      0.90      1439

    accuracy                           0.82      1752
   macro avg       0.68      0.57      0.59      1752
weighted avg       0.79      0.82      0.79      1752



In [ ]:
confusion_matrix(y_test, lasso_logistic_preds_test)

array([[  59,  254],
       [  56, 1383]])

#####  **Daily Mean SO2 Regression (Cross-Validation):**

In [ ]:
endog = np.log(data[data['SO2_above_0'] == 1]['mean_so2'] + 0.001)
scaler = StandardScaler().fit(data[data['SO2_above_0'] == 1].iloc[:,6:318])
exog = np.concatenate((
        scaler.transform(data[data['SO2_above_0'] == 1].iloc[:,6:318]),
        data[data['SO2_above_0'] == 1].iloc[:,318:323],
), axis = 1)
group = data[data['SO2_above_0'] == 1]['id']

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.2, random_state=42, stratify=group)

In [ ]:
lasso_log_mean = Pipeline([("imputer", KNNImputer(n_neighbors=8, weights='uniform')),
                           ("regressor", Lasso(alpha=0.1, max_iter=6000, tol=0.00001))])

m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
  m1b, m2b, m3b, m4b, m5b, m6b, m7b = cross_validate_fit_regression(lasso_log_mean , X_train, y_train)

In [ ]:
print("R2: " + str(np.mean(m1a)))
print("r: " + str(np.mean(m2a)))
print("MAE: " + str(np.mean(m3a)))
print("MedAE: " + str(np.mean(m4a)))
print("MAE (original): " + str(np.mean(m5a)))
print("MedAE (original): " + str(np.mean(m6a)))
print("upper quartile (original): " + str(np.mean(m7a)))

R2: 0.1507482451810397
r: 0.4087495253880659
MAE: 1.1163779408479935
MedAE: 0.8300251010593171
MAE (original): 0.8148962207142929
MedAE (original): 0.19921558668536882
upper quartile (original): 0.44838283068481777


#####  **Daily Mean SO2 Regression (Training & Testing):**

In [ ]:
lasso_log_mean = Pipeline([("imputer", KNNImputer(n_neighbors=8, weights='uniform')),
                           ("regressor", Lasso(alpha=0.1, max_iter=6000, tol=0.00001))])
lasso_log_mean.fit(X_train, y_train)

Pipeline(steps=[('imputer', KNNImputer(n_neighbors=8)),
                ('regressor', Lasso(alpha=0.1, max_iter=6000, tol=1e-05))])

In [ ]:
lasso_log_mean_preds_train = lasso_log_mean.predict(X_train)
lasso_log_mean_preds_test = lasso_log_mean.predict(X_test)

In [ ]:
print("Train R2: " + str(r2_score(y_train, lasso_log_mean_preds_train)))
print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train).flatten(), lasso_log_mean_preds_train)[0]))
print("Train MSE: " + str(mean_squared_error(y_train, lasso_log_mean_preds_train)))
print("Train MAE: " + str(mean_absolute_error(y_train, lasso_log_mean_preds_train)))
print("Train MedAE: " + str(median_absolute_error(y_train, lasso_log_mean_preds_train)))
print("Train MSE (original): " + str(mean_squared_error(np.exp(y_train) - 0.001, np.exp(lasso_log_mean_preds_train) - 0.001)))
print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train) - 0.001, np.exp(lasso_log_mean_preds_train) - 0.001)))
print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train) - 0.001, np.exp(lasso_log_mean_preds_train) - 0.001)))
print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train) - np.exp(lasso_log_mean_preds_train)), 0.75)))
print("\n")
print("Test R2: " + str(r2_score(y_test, lasso_log_mean_preds_test)))
print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test).flatten(), lasso_log_mean_preds_test)[0]))
print("Test MSE: " + str(mean_squared_error(y_test, lasso_log_mean_preds_test)))
print("Test MAE: " + str(mean_absolute_error(y_test, lasso_log_mean_preds_test)))
print("Test MedAE: " + str(median_absolute_error(y_test, lasso_log_mean_preds_test)))
print("Test MSE (original): " + str(mean_squared_error(np.exp(y_test) - 0.001, np.exp(lasso_log_mean_preds_test) - 0.001)))
print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test) - 0.001, np.exp(lasso_log_mean_preds_test) - 0.001)))
print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test) - 0.001, np.exp(lasso_log_mean_preds_test) - 0.001)))
print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test) - np.exp(lasso_log_mean_preds_test)), 0.75)))

Train R2: 0.15287259024661226
Train R: 0.4082265723661725
Train MSE: 2.20332378995343
Train MAE: 1.116357074280986
Train MedAE: 0.825328037950469
Train MSE (original): 5.557832202033876
Train MAE (original): 0.8148530130611585
Train MedAE (original): 0.19907016638564629
Train upper quartile (original): 0.4410981743743934


Test R2: 0.14348321663067753
Test R: 0.39574849226318876
Test MSE: 2.3200010940964138
Test MAE: 1.1526888979525207
Test MedAE: 0.8352462067136561
Test MSE (original): 5.566119896817452
Test MAE (original): 0.8057819017377825
Test MedAE (original): 0.19104292506727247
Test upper quartile (original): 0.42555740525776237


#####  **Daily Max SO2 Regression (Cross-Validation):**

In [ ]:
endog = np.log(data[data['SO2_above_0'] == 1]['max_so2'] + 0.001)
scaler = StandardScaler().fit(data[data['SO2_above_0'] == 1].iloc[:,6:318])
exog = np.concatenate((
        scaler.transform(data[data['SO2_above_0'] == 1].iloc[:,6:318]),
        data[data['SO2_above_0'] == 1].iloc[:,318:323],
), axis = 1)
group = data[data['SO2_above_0'] == 1]['id']

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.2, random_state=42, stratify=group)

In [ ]:
lasso_log_max = Pipeline([("imputer", KNNImputer(n_neighbors=8, weights='uniform')),
                           ("regressor", Lasso(alpha=0.1, max_iter=6000, tol=0.00001))])

m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
  m1b, m2b, m3b, m4b, m5b, m6b, m7b = cross_validate_fit_regression(lasso_log_max , X_train, y_train)

In [ ]:
print("R2: " + str(np.mean(m1a)))
print("r: " + str(np.mean(m2a)))
print("MAE: " + str(np.mean(m3a)))
print("MedAE: " + str(np.mean(m4a)))
print("MAE (original): " + str(np.mean(m5a)))
print("MedAE (original): " + str(np.mean(m6a)))
print("upper quartile (original): " + str(np.mean(m7a)))

R2: 0.13863163485599114
r: 0.39582351829566825
MAE: 1.137750330270834
MedAE: 0.9855915915487634
MAE (original): 3.6180353457139276
MedAE (original): 0.8361312318641596
upper quartile (original): 2.1003414300845376


#####  **Daily Max SO2 Regression (Training & Testing):**

In [ ]:
lasso_log_max = Pipeline([("imputer", KNNImputer(n_neighbors=8, weights='uniform')),
                          ("regressor", Lasso(alpha=0.1, max_iter=6000, tol=0.00001))])
lasso_log_max.fit(X_train, y_train)

Pipeline(steps=[('imputer', KNNImputer(n_neighbors=8)),
                ('regressor', Lasso(alpha=0.1, max_iter=6000, tol=1e-05))])

In [ ]:
lasso_log_max_preds_train = lasso_log_max.predict(X_train)
lasso_log_max_preds_test = lasso_log_max.predict(X_test)

In [ ]:
print("Train R2: " + str(r2_score(y_train, lasso_log_max_preds_train)))
print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train).flatten(), lasso_log_max_preds_train)[0]))
print("Train MSE: " + str(mean_squared_error(y_train, lasso_log_max_preds_train)))
print("Train MAE: " + str(mean_absolute_error(y_train, lasso_log_max_preds_train)))
print("Train MedAE: " + str(median_absolute_error(y_train, lasso_log_max_preds_train)))
print("Train MSE (original): " + str(mean_squared_error(np.exp(y_train) - 0.001, np.exp(lasso_log_max_preds_train) - 0.001)))
print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train) - 0.001, np.exp(lasso_log_max_preds_train) - 0.001)))
print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train) - 0.001, np.exp(lasso_log_max_preds_train) - 0.001)))
print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train) - np.exp(lasso_log_max_preds_train)), 0.75)))
print("\n")
print("Test R2: " + str(r2_score(y_test, lasso_log_max_preds_test)))
print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test).flatten(), lasso_log_max_preds_test)[0]))
print("Test MSE: " + str(mean_squared_error(y_test, lasso_log_max_preds_test)))
print("Test MAE: " + str(mean_absolute_error(y_test, lasso_log_max_preds_test)))
print("Test MedAE: " + str(median_absolute_error(y_test, lasso_log_max_preds_test)))
print("Test MSE (original): " + str(mean_squared_error(np.exp(y_test) - 0.001, np.exp(lasso_log_max_preds_test) - 0.001)))
print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test) - 0.001, np.exp(lasso_log_max_preds_test) - 0.001)))
print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test) - 0.001, np.exp(lasso_log_max_preds_test) - 0.001)))
print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test) - np.exp(lasso_log_max_preds_test)), 0.75)))

Train R2: 0.13846005376050152
Train R: 0.39122792782896315
Train MSE: 1.9834102003760028
Train MAE: 1.1389703330931107
Train MedAE: 0.9892191531122974
Train MSE (original): 121.46340730511602
Train MAE (original): 3.6196614360544954
Train MedAE (original): 0.8337646743498995
Train upper quartile (original): 2.0789178599159417


Test R2: 0.1389887859565262
Test R: 0.39516820205720293
Test MSE: 2.0050801556247535
Test MAE: 1.1531252708560813
Test MedAE: 0.9978067120923934
Test MSE (original): 142.04780170668744
Test MAE (original): 3.559671265033243
Test MedAE (original): 0.8277682906044272
Test upper quartile (original): 1.8519437926380387


####  **CONTROL 2) HGBM： only Meteorological and Land-Use Land-Cover features**

#####  **Binary Classification (Cross-Validation):**

In [ ]:
y = data['SO2_above_0']
X = data.iloc[:,6:317]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
binary_gbm = HistGradientBoostingClassifier(max_iter = 5000, learning_rate = 0.001,
                                            max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                            random_state=42,
                                            class_weight={0: 1 / (Counter(data['SO2_above_0'])[0] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0])),
                                                          1: 1 / (Counter(data['SO2_above_0'])[1] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0]))})

accuracy, precision, recall, f1score = cross_validate_fit_binary_classification(binary_gbm , X_train, y_train)

In [ ]:
print(np.mean([accuracy[n][0] for n in range(10)]))
print(np.mean([precision[n][0] for n in range(10)]))
print(np.mean([recall[n][0] for n in range(10)]))
print(np.mean([f1score[n][0] for n in range(10)]))

0.7634169553698797
0.8350738856609181
0.7634169553698797
0.7855050897634348


#####  **Binary Classification (Training & Testing):**

In [ ]:
binary_gbm_control = HistGradientBoostingClassifier(max_iter = 5000, learning_rate = 0.001,
                                                    max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                                    random_state=42,
                                                    class_weight={0: 1 / (Counter(data['SO2_above_0'])[0] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0])),
                                                                  1: 1 / (Counter(data['SO2_above_0'])[1] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0]))})
binary_gbm_control.fit(X_train, y_train)

HistGradientBoostingClassifier(class_weight={0: 5.597444089456869,
                                             1: 1.2175121612230715},
                               learning_rate=0.001, max_depth=5, max_iter=5000,
                               random_state=42)

In [ ]:
binary_control_preds_train = binary_gbm_control.predict(X_train)
binary_control_preds_test = binary_gbm_control.predict(X_test)

In [ ]:
print(classification_report(y_train, binary_control_preds_train))

              precision    recall  f1-score   support

           0       0.52      0.92      0.66      1252
           1       0.98      0.81      0.89      5756

    accuracy                           0.83      7008
   macro avg       0.75      0.87      0.77      7008
weighted avg       0.90      0.83      0.85      7008



In [ ]:
confusion_matrix(y_train, binary_control_preds_train)

array([[1153,   99],
       [1084, 4672]])

In [ ]:
print(classification_report(y_test, binary_control_preds_test))

              precision    recall  f1-score   support

           0       0.39      0.73      0.51       313
           1       0.93      0.75      0.83      1439

    accuracy                           0.75      1752
   macro avg       0.66      0.74      0.67      1752
weighted avg       0.83      0.75      0.77      1752



In [ ]:
confusion_matrix(y_test, binary_control_preds_test)

array([[ 230,   83],
       [ 361, 1078]])

#####  **Daily Mean SO2 Regression (Cross-Validation):**

In [ ]:
endog = np.log(data[data['SO2_above_0'] == 1]['mean_so2'] + 0.001)
scaler = StandardScaler().fit(data[data['SO2_above_0'] == 1].iloc[:,6:318])
exog = np.concatenate((
        scaler.transform(data[data['SO2_above_0'] == 1].iloc[:,6:318]),
        data[data['SO2_above_0'] == 1].iloc[:,318:323],
), axis = 1)
group = data[data['SO2_above_0'] == 1]['id']

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.2, random_state=42, stratify=group)

In [ ]:
gbmlogmean = HistGradientBoostingRegressor(max_iter = 5000,
                                           learning_rate = 0.001,
                                           max_depth = 5,
                                           max_leaf_nodes = 2 ** 5 - 1,
                                           random_state=42)

m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
  m1b, m2b, m3b, m4b, m5b, m6b, m7b = cross_validate_fit_regression(gbmlogmean , X_train, y_train)

In [ ]:
print("R2: " + str(np.mean(m1a)))
print("r: " + str(np.mean(m2a)))
print("MAE: " + str(np.mean(m3a)))
print("MedAE: " + str(np.mean(m4a)))
print("MAE (original): " + str(np.mean(m5a)))
print("MedAE (original): " + str(np.mean(m6a)))
print("upper quartile (original): " + str(np.mean(m7a)))

R2: 0.3610050739766022
r: 0.6129978196659439
MAE: 0.9705195046383699
MedAE: 0.7367856474259866
MAE (original): 0.7311278576341758
MedAE (original): 0.17890754246962404
upper quartile (original): 0.4517319296428453


#####  **Daily Mean SO2 Regression (Training & Testing):**

In [ ]:
gbmlogmean_control = HistGradientBoostingRegressor(max_iter = 5000, learning_rate = 0.001,
                                                  max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                                  random_state=42)
gbmlogmean_control.fit(X_train, y_train)

HistGradientBoostingRegressor(learning_rate=0.001, max_depth=5, max_iter=5000,
                              random_state=42)

In [ ]:
gbmlogmean_control_preds_train = gbmlogmean_control.predict(X_train)
gbmlogmean_control_preds_test = gbmlogmean_control.predict(X_test)

In [ ]:
print("Train R2: " + str(r2_score(y_train, gbmlogmean_control_preds_train)))
print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train).flatten(), gbmlogmean_control_preds_train)[0]))
print("Train MSE: " + str(mean_squared_error(y_train, gbmlogmean_control_preds_train)))
print("Train MAE: " + str(mean_absolute_error(y_train, gbmlogmean_control_preds_train)))
print("Train MedAE: " + str(median_absolute_error(y_train, gbmlogmean_control_preds_train)))
print("Train MAE (original): " + str(mean_squared_error(np.exp(y_train) - 0.001, np.exp(gbmlogmean_control_preds_train) - 0.001)))
print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmean_control_preds_train) - 0.001)))
print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmean_control_preds_train) - 0.001)))
print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train) - np.exp(gbmlogmean_control_preds_train)), 0.75)))
print("\n")
print("Test R2: " + str(r2_score(y_test, gbmlogmean_control_preds_test)))
print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test).flatten(), gbmlogmean_control_preds_test)[0]))
print("Test MSE: " + str(mean_squared_error(y_test, gbmlogmean_control_preds_test)))
print("Test MAE: " + str(mean_absolute_error(y_test, gbmlogmean_control_preds_test)))
print("Test MedAE: " + str(median_absolute_error(y_test, gbmlogmean_control_preds_test)))
print("Test MAE (original): " + str(mean_squared_error(np.exp(y_test) - 0.001, np.exp(gbmlogmean_control_preds_test) - 0.001)))
print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmean_control_preds_test) - 0.001)))
print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmean_control_preds_test) - 0.001)))
print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test) - np.exp(gbmlogmean_control_preds_test)), 0.75)))

Train R2: 0.5368301264013555
Train R: 0.7699660299922577
Train MSE: 1.2046749869499607
Train MAE: 0.8266513643840013
Train MedAE: 0.6300612481581418
Train MAE (original): 4.331417141292759
Train MAE (original): 0.6728999844494842
Train MedAE (original): 0.14851535692377726
Train upper quartile (original): 0.3773149618863065


Test R2: 0.37306456415363665
Test R: 0.6246709015027246
Test MSE: 1.6981464056895326
Test MAE: 0.9798760484403296
Test MedAE: 0.7248643773732115
Test MAE (original): 4.672447199992607
Test MAE (original): 0.717112555946295
Test MedAE (original): 0.16798696779074623
Test upper quartile (original): 0.40430344415175845


#####  **Daily Max SO2 Regression (Cross-Validation):**

In [ ]:
endog = np.log(data[data['SO2_above_0'] == 1]['max_so2'] + 0.001)
scaler = StandardScaler().fit(data[data['SO2_above_0'] == 1].iloc[:,6:318])
exog = np.concatenate((
        scaler.transform(data[data['SO2_above_0'] == 1].iloc[:,6:318]),
        data[data['SO2_above_0'] == 1].iloc[:,318:323],
), axis = 1)
group = data[data['SO2_above_0'] == 1]['id']

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.2, random_state=42, stratify=group)

In [ ]:
gbmlogmean = HistGradientBoostingRegressor(max_iter = 5000,
                                           learning_rate = 0.001,
                                           max_depth = 5,
                                           max_leaf_nodes = 2 ** 5 - 1,
                                           random_state=42)

m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
  m1b, m2b, m3b, m4b, m5b, m6b, m7b = cross_validate_fit_regression(gbmlogmean , X_train, y_train)

In [ ]:
print("R2: " + str(np.mean(m1a)))
print("r: " + str(np.mean(m2a)))
print("MAE: " + str(np.mean(m3a)))
print("MedAE: " + str(np.mean(m4a)))
print("MAE (original): " + str(np.mean(m5a)))
print("MedAE (original): " + str(np.mean(m6a)))
print("upper quartile (original): " + str(np.mean(m7a)))

R2: 0.39294678881874345
r: 0.6405433313951292
MAE: 0.9260604974411522
MedAE: 0.7557678870364949
MAE (original): 3.246880789714261
MedAE (original): 0.7272897597277324
upper quartile (original): 2.0271322130802734


#####  **Daily Max SO2 Regression (Training & Testing):**

In [ ]:
gbmlogmax_control = HistGradientBoostingRegressor(max_iter = 5000, learning_rate = 0.001,
                                                  max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                                  random_state=42)
gbmlogmax_control.fit(X_train, y_train)

HistGradientBoostingRegressor(learning_rate=0.001, max_depth=5, max_iter=5000,
                              random_state=42)

In [ ]:
gbmlogmax_control_preds_train = gbmlogmax_control.predict(X_train)
gbmlogmax_control_preds_test = gbmlogmax_control.predict(X_test)

In [ ]:
print("Train R2: " + str(r2_score(y_train, gbmlogmax_control_preds_train)))
print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train).flatten(), gbmlogmax_control_preds_train)[0]))
print("Train MSE: " + str(mean_squared_error(y_train, gbmlogmax_control_preds_train)))
print("Train MAE: " + str(mean_absolute_error(y_train, gbmlogmax_control_preds_train)))
print("Train MedAE: " + str(median_absolute_error(y_train, gbmlogmax_control_preds_train)))
print("Train MSE (original): " + str(mean_squared_error(np.exp(y_train) - 0.001, np.exp(gbmlogmax_control_preds_train) - 0.001)))
print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmax_control_preds_train) - 0.001)))
print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmax_control_preds_train) - 0.001)))
print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train) - np.exp(gbmlogmax_control_preds_train)), 0.75)))
print("\n")
print("Test R2: " + str(r2_score(y_test, gbmlogmax_control_preds_test)))
print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test).flatten(), gbmlogmax_control_preds_test)[0]))
print("Test MSE: " + str(mean_squared_error(y_test, gbmlogmax_control_preds_test)))
print("Test MAE: " + str(mean_absolute_error(y_test, gbmlogmax_control_preds_test)))
print("Test MedAE: " + str(median_absolute_error(y_test, gbmlogmax_control_preds_test)))
print("Test MSE (original): " + str(mean_squared_error(np.exp(y_test) - 0.001, np.exp(gbmlogmax_control_preds_test) - 0.001)))
print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmax_control_preds_test) - 0.001)))
print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmax_control_preds_test) - 0.001)))
print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test) - np.exp(gbmlogmax_control_preds_test)), 0.75)))

Train R2: 0.5591104870509616
Train R: 0.7814019703815241
Train MSE: 1.0150019868943363
Train MAE: 0.7880973251401575
Train MedAE: 0.64460821133303
Train MSE (original): 104.14630095409971
Train MAE (original): 3.006345130889141
Train MedAE (original): 0.5849721186131842
Train upper quartile (original): 1.6497116795164788


Test R2: 0.39958067168821776
Test R: 0.6455019029422159
Test MSE: 1.3982267136775202
Test MAE: 0.93934434975434
Test MedAE: 0.7722671068511082
Test MSE (original): 132.13614215967482
Test MAE (original): 3.227593159418506
Test MedAE (original): 0.7221773817801661
Test upper quartile (original): 1.8005347357789319


####  **FINAL 3) HGBM： Full features with Spatiotemporally-lagged Surface Monitored SO2**

#####  **Binary Classification (Cross-Validation):**

In [ ]:
y = data['SO2_above_0']
X = np.array(data.iloc[:,6:-1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
binary_gbm = HistGradientBoostingClassifier(max_iter = 5000, learning_rate = 0.001,
                                            max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                            random_state=42,
                                            class_weight={0: 1 / (Counter(data['SO2_above_0'])[0] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0])),
                                                          1: 1 / (Counter(data['SO2_above_0'])[1] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0]))})

accuracy, precision, recall, f1score = cross_validate_fit_binary_classification(binary_gbm , X_train, y_train)

In [ ]:
print(np.mean([accuracy[n][0] for n in range(10)]))
print(np.mean([precision[n][0] for n in range(10)]))
print(np.mean([recall[n][0] for n in range(10)]))
print(np.mean([f1score[n][0] for n in range(10)]))

0.8327615651110657
0.8647645435105273
0.8327615651110657
0.8433890111154948


#####  **Binary Classification (Training & Testing):**

In [ ]:
binary_gbm = HistGradientBoostingClassifier(max_iter = 5000, learning_rate = 0.001,
                                            max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                            random_state=42,
                                            class_weight={0: 1 / (Counter(data['SO2_above_0'])[0] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0])),
                                                          1: 1 / (Counter(data['SO2_above_0'])[1] / (Counter(data['SO2_above_0'])[1] + Counter(data['SO2_above_0'])[0]))})
binary_gbm.fit(X_train, y_train)

HistGradientBoostingClassifier(class_weight={0: 5.597444089456869,
                                             1: 1.2175121612230715},
                               learning_rate=0.001, max_depth=5, max_iter=5000,
                               random_state=42)

In [ ]:
binary_preds_train = binary_gbm.predict(X_train)
binary_preds_test = binary_gbm.predict(X_test)

In [ ]:
print(classification_report(y_train, binary_preds_train))

              precision    recall  f1-score   support

           0       0.65      0.97      0.78      1252
           1       0.99      0.89      0.94      5756

    accuracy                           0.90      7008
   macro avg       0.82      0.93      0.86      7008
weighted avg       0.93      0.90      0.91      7008



In [ ]:
confusion_matrix(y_train, binary_preds_train)

array([[1218,   34],
       [ 646, 5110]])

In [ ]:
print(classification_report(y_test, binary_preds_test))

              precision    recall  f1-score   support

           0       0.51      0.77      0.62       313
           1       0.94      0.84      0.89      1439

    accuracy                           0.83      1752
   macro avg       0.73      0.81      0.75      1752
weighted avg       0.87      0.83      0.84      1752



In [ ]:
confusion_matrix(y_test, binary_preds_test)

array([[ 242,   71],
       [ 230, 1209]])

#####  **Daily Mean SO2 Regression (Cross-Validation):**

In [ ]:
endog = np.log(data[data['SO2_above_0'] == 1]['mean_so2'] + 0.001)
scaler = StandardScaler().fit(data[data['SO2_above_0'] == 1].iloc[:,6:318])
exog = np.concatenate((
        scaler.transform(data[data['SO2_above_0'] == 1].iloc[:,6:318]),
        data[data['SO2_above_0'] == 1].iloc[:,318:359],
), axis = 1)
group = data[data['SO2_above_0'] == 1]['id']

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.2, random_state=42, stratify=group)

In [ ]:
gbmlogmean = HistGradientBoostingRegressor(max_iter = 5000,
                                           learning_rate = 0.001,
                                           max_depth = 5,
                                           max_leaf_nodes = 2 ** 5 - 1,
                                           random_state=42)

m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
  m1b, m2b, m3b, m4b, m5b, m6b, m7b = cross_validate_fit_regression(gbmlogmean , X_train, y_train)

In [ ]:
print("R2: " + str(np.mean(m1a)))
print("r: " + str(np.mean(m2a)))
print("MAE: " + str(np.mean(m3a)))
print("MedAE: " + str(np.mean(m4a)))
print("MAE (original): " + str(np.mean(m5a)))
print("MedAE (original): " + str(np.mean(m6a)))
print("upper quartile (original): " + str(np.mean(m7a)))

R2: 0.4516681437514468
r: 0.6780875167875762
MAE: 0.8753963710191138
MedAE: 0.619042323135775
MAE (original): 0.6370954662297749
MedAE (original): 0.15566597208343286
upper quartile (original): 0.39112558145850745


#####  **Daily Mean SO2 Regression (Training & Testing):**

In [ ]:
gbmlogmean = HistGradientBoostingRegressor(max_iter = 5000, learning_rate = 0.001,
                                           max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                           random_state=42)
gbmlogmean.fit(X_train, y_train)

HistGradientBoostingRegressor(learning_rate=0.001, max_depth=5, max_iter=5000,
                              random_state=42)

In [ ]:
gbmlogmean_preds_train = gbmlogmean.predict(X_train)
gbmlogmean_preds_test = gbmlogmean.predict(X_test)

In [ ]:
####  max_iter = 5000, learning_rate = 0.001, max_depth = 5, max_leaf_nodes = 2 ** 5 - 1
####  v4:  decay_alpha = 0.1, station distance = 10 km,  standardization RS data + log-transformation SO2 data

print("Train R2: " + str(r2_score(y_train, gbmlogmean_preds_train)))
print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train).flatten(), gbmlogmean_preds_train)[0]))
print("Train MSE: " + str(mean_squared_error(y_train, gbmlogmean_preds_train)))
print("Train MAE: " + str(mean_absolute_error(y_train, gbmlogmean_preds_train)))
print("Train MedAE: " + str(median_absolute_error(y_train, gbmlogmean_preds_train)))
print("Train MSE (original): " + str(mean_squared_error(np.exp(y_train) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001)))
print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001)))
print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmean_preds_train) - 0.001)))
print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train) - np.exp(gbmlogmean_preds_train)), 0.75)))
print("\n")
print("Test R2: " + str(r2_score(y_test, gbmlogmean_preds_test)))
print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test).flatten(), gbmlogmean_preds_test)[0]))
print("Test MSE: " + str(mean_squared_error(y_test, gbmlogmean_preds_test)))
print("Test MAE: " + str(mean_absolute_error(y_test, gbmlogmean_preds_test)))
print("Test MedAE: " + str(median_absolute_error(y_test, gbmlogmean_preds_test)))
print("Test MSE (original): " + str(mean_squared_error(np.exp(y_test) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001)))
print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001)))
print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmean_preds_test) - 0.001)))
print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test) - np.exp(gbmlogmean_preds_test)), 0.75)))

Train R2: 0.6289069325385701
Train R: 0.8117141012503041
Train MSE: 0.9651891491299863
Train MAE: 0.7212546652950914
Train MedAE: 0.5074834802975416
Train MSE (original): 3.068219036822679
Train MAE (original): 0.5526843964512465
Train MedAE (original): 0.12449872296090819
Train upper quartile (original): 0.3211331410126239


Test R2: 0.4583985696755243
Test R: 0.680966368681645
Test MSE: 1.4670067596038778
Test MAE: 0.8856843882159745
Test MedAE: 0.6193564505781279
Test MSE (original): 3.4382369691488037
Test MAE (original): 0.6243305007070176
Test MedAE (original): 0.147856852401518
Test upper quartile (original): 0.40674268535485725


#####  **Daily Max SO2 Regression (Cross-Validation):**

In [ ]:
endog = np.log(data[data['SO2_above_0'] == 1]['max_so2'] + 0.001)
scaler = StandardScaler().fit(data[data['SO2_above_0'] == 1].iloc[:,6:318])
exog = np.concatenate((
        scaler.transform(data[data['SO2_above_0'] == 1].iloc[:,6:318]),
        data[data['SO2_above_0'] == 1].iloc[:,318:359],
), axis = 1)
group = data[data['SO2_above_0'] == 1]['id']

X_train, X_test, y_train, y_test = train_test_split(exog, endog, test_size=0.2, random_state=42, stratify=group)

In [ ]:
gbmlogmax = HistGradientBoostingRegressor(max_iter = 5000,
                                          learning_rate = 0.001,
                                          max_depth = 5,
                                          max_leaf_nodes = 2 ** 5 - 1,
                                          random_state=42)

m1a, m2a, m3a, m4a, m5a, m6a, m7a, \
  m1b, m2b, m3b, m4b, m5b, m6b, m7b = cross_validate_fit_regression(gbmlogmean , X_train, y_train)

In [ ]:
print("R2: " + str(np.mean(m1a)))
print("r: " + str(np.mean(m2a)))
print("MAE: " + str(np.mean(m3a)))
print("MedAE: " + str(np.mean(m4a)))
print("MAE (original): " + str(np.mean(m5a)))
print("MedAE (original): " + str(np.mean(m6a)))
print("upper quartile (original): " + str(np.mean(m7a)))

R2: 0.4354298986371301
r: 0.6667809397371164
MAE: 0.8777700081438207
MedAE: 0.7014882666563196
MAE (original): 3.051195567134058
MedAE (original): 0.6703421032854444
upper quartile (original): 2.066076048741021


#####  **Daily Max SO2 Regression (Training & Testing):**

In [ ]:
gbmlogmax = HistGradientBoostingRegressor(max_iter = 5000, learning_rate = 0.001,
                                          max_depth = 5, max_leaf_nodes = 2 ** 5 - 1,
                                          random_state=42)
gbmlogmax.fit(X_train, y_train)

HistGradientBoostingRegressor(learning_rate=0.001, max_depth=5, max_iter=5000,
                              random_state=42)

In [ ]:
gbmlogmax_preds_train = gbmlogmax.predict(X_train)
gbmlogmax_preds_test = gbmlogmax.predict(X_test)

In [ ]:
print("Train R2: " + str(r2_score(y_train, gbmlogmax_preds_train)))
print("Train R: " + str(scipy.stats.pearsonr(np.array(y_train).flatten(), gbmlogmax_preds_train)[0]))
print("Train MSE: " + str(mean_squared_error(y_train, gbmlogmax_preds_train)))
print("Train MAE: " + str(mean_absolute_error(y_train, gbmlogmax_preds_train)))
print("Train MedAE: " + str(median_absolute_error(y_train, gbmlogmax_preds_train)))
print("Train MSE (original): " + str(mean_squared_error(np.exp(y_train) - 0.001, np.exp(gbmlogmax_preds_train) - 0.001)))
print("Train MAE (original): " + str(mean_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmax_preds_train) - 0.001)))
print("Train MedAE (original): " + str(median_absolute_error(np.exp(y_train) - 0.001, np.exp(gbmlogmax_preds_train) - 0.001)))
print("Train upper quartile (original): " + str(np.quantile(abs(np.exp(y_train) - np.exp(gbmlogmax_preds_train)), 0.75)))
print("\n")
print("Test R2: " + str(r2_score(y_test, gbmlogmax_preds_test)))
print("Test R: " + str(scipy.stats.pearsonr(np.array(y_test).flatten(), gbmlogmax_preds_test)[0]))
print("Test MSE: " + str(mean_squared_error(y_test, gbmlogmax_preds_test)))
print("Test MAE: " + str(mean_absolute_error(y_test, gbmlogmax_preds_test)))
print("Test MedAE: " + str(median_absolute_error(y_test, gbmlogmax_preds_test)))
print("Test MSE (original): " + str(mean_squared_error(np.exp(y_test) - 0.001, np.exp(gbmlogmax_preds_test) - 0.001)))
print("Test MAE (original): " + str(mean_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmax_preds_test) - 0.001)))
print("Test MedAE (original): " + str(median_absolute_error(np.exp(y_test) - 0.001, np.exp(gbmlogmax_preds_test) - 0.001)))
print("Test upper quartile (original): " + str(np.quantile(abs(np.exp(y_test) - np.exp(gbmlogmax_preds_test)), 0.75)))

Train R2: 0.6161712019903938
Train R: 0.8062346926606283
Train MSE: 0.8836386014290316
Train MAE: 0.7264585065666358
Train MedAE: 0.578059123347066
Train MSE (original): 92.22006696352874
Train MAE (original): 2.687982191918779
Train MedAE (original): 0.5226432617851546
Train upper quartile (original): 1.5943047736903757


Test R2: 0.4349923916982371
Test R: 0.663623977058623
Test MSE: 1.315761658739171
Test MAE: 0.893742649699439
Test MedAE: 0.7354111383772052
Test MSE (original): 125.03010399048397
Test MAE (original): 3.0245378791448645
Test MedAE (original): 0.6916314552187157
Test upper quartile (original): 1.8216257959085151
